# <center> <h1> Testing </h1> </center>

<img src="../images/tests.png" width="200">

In [ ]:
get_ipython().magic(u'matplotlib inline')
%run -i ../utils/credentials.py
%run -i ../utils/imports.py
%run -i ../utils/plots.py
%run -i ../utils/stats.py

In [ ]:
import string
from numpy.random import choice, sample
import scipy.stats as st

# Problème 
### Test d'indépendance : montrer que la taille d'échantillon compte

La suite de questions qui suivent ont pour objectif de montrer que le test d'indépendance du $\chi^2$ n'est pas approprié lorsque la taille de l'échantillon est faible et qu'un test de Fisher peut être envisagé à la place... à certaines conditions.

Les théorèmes utilisée pour réaliser des tests (théorème central limite) étant souvent asymptotiques, il faut rester prudent lorsque les effectifs sont faibles. 

Pour s'en convaincre on va réaliser quelques tests sur des échantillon de taille différents.

## Question 1
1/ Construire un fonction renvoyant un DataFrame à deux colonnes, dont chacune est une variable aléatoires qualitative (les modalités peuvent être prises parmis les lettres de l'alphabet par exemple). Dans cette question les deux variables sont indépendantes, i.e le tirage de l'une et de l'autre sont réalisés séparemment.

Les **arguments** de la fonction sont : 
* p1 : les probabilités donnant la répartition des modalités pour la première variable, la somme doit faire 1 et la taille de cette liste donne le nombre de modalités pour cette variable.
* p2 : les probabilités donnant la répartition des modalités pour la deuxième variable, la somme doit faire 1 et la taille de cette liste donne le nombre de modalités pour cette variable.
* sample_size : la taille de l'échantillon que l'on créer

Le **dataFrame** de sortie : 
* les colonnes sont nommées : "Y1" et "Y2"

2/ Calculer la matrice de contingence d'un tirage de deux variables qualitatives indépendantes ayant les caractéristiqeques suivantes.

### Réponse 1

In [ ]:
def simu_sample_indep_Y1_Y2(p1, p2, sample_size):
    """
    simu_sample_indep_Y1_Y2([0.1,0.1,0.2,0.6], [0.5,0.5], 10)
    """
    choices1 = list(string.ascii_lowercase)[0:len(p1)]
    choices2 = list(string.ascii_lowercase)[-len(p2):]
    data_Y1 = choice(choices1, sample_size, p = p1)
    data_Y2 = choice(choices2, sample_size, p = p2)
    df = pd.DataFrame({"Y1":data_Y1,"Y2":data_Y2})
    return df

In [ ]:
p1 = [0.1,0.1,0.2,0.6]
p2 = [0.5,0.5]
sample_size = 1000

In [ ]:
df = simu_sample_indep_Y1_Y2(p1, p2, sample_size)
cont = pd.crosstab(df.Y1,df.Y2)
cont

## Question 2
1/ Réaliser un test d'indépendance du $\chi^2$ à partir de la table de contingence et vérifier que le test est bien symétrique (les deux colonnes peuvent être inversées)

2/ Est-ce que l'on rejette ou non l'hypothèse d'indépendance ? Si oui que peut on en conclure ?

### Réponse 2

In [ ]:
st_chi2, st_p, st_dof, st_exp = st.chi2_contingency(cont)
print("valeur de la statistique : {}".format(st_chi2))
print("valeur de la p_value : {}".format(st_p))

In [ ]:
st_chi2, st_p, st_dof, st_exp = st.chi2_contingency(cont.transpose())
print("valeur de la statistique : {}".format(st_chi2))
print("valeur de la p_value : {}".format(st_p))

On rejette l'hypothèse d'indépendance à 5% si la p_value est inférieure à 5%. Comme nous avons réalisé une simulation nous savons que les variables sont bien indépendantes. Si nous rejetons l'hypothèse nulle cela signifie que nous sommes tombé sur les 5% de cas où l'on rejette à tort. Dans ce cas faites retourner le test plusieurs fois. On doit l'accépter dans la plupart des cas.

## Question 3
On utilise la fonction suivante pour construire deux vecteurs qualitatifs non indépendantes.

In [ ]:
def simu_sample_notindep_Y1_Y2(p1, p2, sample_size):
    """
    Exemple : 
        simu_sample_notindep_Y1_Y2(p1 = [0.2,0.8], 
                                   p2 = [0.2,0.8,0.5,0.5],
                                   sample_size = 10)
    """    
    
    assert((len(p2)%len(p1))==0), "len(p2) is not a multiple of len(p1)"
    
    # The first column is sampled
    choices1 = list(string.ascii_lowercase)[0:len(p1)]
    data_Y1 = choice(choices1, sample_size, p = p1)
    df = pd.DataFrame({"Y1":data_Y1})
    
    len_1 = len(choices1)
    choices2 = list(string.ascii_lowercase)[0:(len(p2)//len_1)]
    
    # For each modality of the first column, the second is sampled with a different probability distribution
    for i in range(len(choices1)):
        size = np.sum(df["Y1"]==choices1[i])
        Y_2_i = choice(choices2, size, p = p2[i*(len(p2)//len_1):(1+i)*(len(p2)//len_1)])
        df.loc[df["Y1"]==choices1[i],"Y2"] = Y_2_i
    
    return df

1/ Créer une fonction pour répéter un grand nombre de fois un test du $\chi^2$ sur un grand nombre d'échantillons de variables qualitatives non appariées. Utiliser les paramètres suivants. 

In [ ]:
sample_size = 200
p1 = [0.4,0.5,0.1]
p2 = [0.1,0.9,0.2,0.8,0.5,0.5]

2/ Visualiser les résultats à l'aide de boxplots

### Réponse 3

In [ ]:
def repeat_experience(simu_func, p1, p2, sample_size, nb_repeat = 1000):
    """
    Exemple :
        repeat_experience(simu_sample_notindep_Y1_Y2, 
                         p1 = [0.1,0.9],
                         p2 = [0.1, 0.9, 0.5, 0.5],
                         sample_size =100,
                         nb_repeat = 10)
    """
    pvals=[]
    T_obs = []
    
    for ii in range(nb_repeat):
        df = simu_func(p1,p2,sample_size)
        cont = pd.crosstab(df.Y1,df.Y2)
        st_chi2, st_p, st_dof, st_exp = st.chi2_contingency(cont)
        T_obs.append(st_chi2)
        pvals.append(round(st_p,5))
    
    return T_obs, pvals

In [ ]:
T_obs, pvals = repeat_experience(simu_sample_notindep_Y1_Y2, p1, p2, sample_size)

In [ ]:
plt.boxplot(pvals)
plt.show()

# Question 4
Faire la même chose en faisant varier la taille des échantillons comme suit.

In [ ]:
sizes = [10,30,50,100,200]

Intérpréter les résultats.

### Réponse 4

In [ ]:
def repeat_experience_several_sample_size(sizes, simu_func, p1, p2):
    results = {}
    for sample_size in sizes:
        T_obs, pvals = repeat_experience(simu_sample_notindep_Y1_Y2, p1, p2, sample_size)
        results["p_values_{}".format(sample_size)] = pvals
        results["T_obs_{}".format(sample_size)] = T_obs    
    return results

In [ ]:
result_notindep_chi2 = repeat_experience_several_sample_size(sizes, 
                                                        simu_sample_notindep_Y1_Y2, 
                                                        p1,
                                                        p2)

In [ ]:
result_notindep_pvals_chi2 = pd.DataFrame(result_notindep_chi2).iloc[:,[0,2,4,6,8]].transpose()

In [ ]:
plt.boxplot(result_notindep_pvals_chi2)
plt.xticks([1,2,3,4,5], sizes, fontsize = 15)
plt.yticks(fontsize = 15)
plt.ylabel("p-value", fontsize = 15)
plt.xlabel("Taille des échantillons", fontsize = 15)
plt.title("Répartition des p-value pour un test d'indépendance du $\chi2$ \n\
            pour 1000 échantillons\n\
            en fonction de la taille des échantillons", fontsize = 25)
plt.plot((0,7),(0.05,0.05))
plt.show()

Lorsque l'échantillon est petit on va accépter souvent l'hypothèse d'indépendance alors que les de vecteur sont liés. La puissance du test dépend donc de la taille de l'échantillon

# Question 5
Réaliser la même expérience en réalisant des tests de Fisher. Pour cela on n'utilisera que deux modalités car la fonction ne supporte le test que pour des matrices $2\times2$.

### Réponse 5

In [ ]:
def fill_2_2_crosstab(cont):
    """
    Exemple :
        test_df = pd.DataFrame({"Y1":["a","a"],"Y2":["a","a"]})
        test_cont = pd.crosstab(test_df.Y1,test_df.Y2)
        fill_2_2_crosstab(test_cont)
    """
    cont_inter = pd.DataFrame({"a":[0,0], "b":[0,0]}, index=["a","b"])
    for i in cont.columns :
        for j in cont.index :
            cont_inter.loc[j,i] = cont.loc[j,i]
    return cont_inter

In [ ]:
def repeat_experience(simu_func, p1, p2, sample_size, test_type="chi2", nb_repeat = 1000):
    """
    Exemple : 
    _, pvals = repeat_experience(simu_func = simu_sample_notindep_Y1_Y2, 
                                 p1 = [0.4, 0.6],
                                 p2 = [0.4, 0.6, 0.5, 0.5],
                                 sample_size = 10,
                                 test_type="fisher",
                                 nb_repeat = 3)
    """
    pvals=[]
    T_obs = []
    
    for ii in range(nb_repeat):
        # Make a simulation of the data
        df = simu_func(p1,p2,sample_size)
        cont = pd.crosstab(df.Y1,df.Y2)
        
        # If the cross-tab size is not (2,2) fill the missing values with 0.
        if cont.shape!=(2,2):
            cont = fill_2_2_crosstab(cont)
        
        # Make the test with the chosen method
        if test_type=="chi2":
            st_chi2, st_p, st_dof, st_exp = st.chi2_contingency(cont)
            T_obs.append(st_chi2)
            pvals.append(round(st_p,5))    
        elif (test_type=="fisher"):
            pvals.append(st.fisher_exact(cont, alternative='two-sided')[1])
    
    return T_obs, pvals

In [ ]:
def repeat_experience_several_sample_size(sample_sizes, simu_func, p1, p2, test_type="chi2", nb_repeat = 1000):
    """
    Exemple :
    repeat_experience_several_sample_size(sample_sizes = [20,30],
                                          simu_func = simu_sample_notindep_Y1_Y2, 
                                          p1 = [0.4, 0.6],
                                          p2 = [0.4, 0.6, 0.5, 0.5],
                                          test_type = "fisher", 
                                          nb_repeat = 3)
    """
    results = {}
    for sample_size in sample_sizes:
        
        # repeat the experience for the sample_size
        T_obs, pvals = repeat_experience(simu_sample_notindep_Y1_Y2, p1, p2, sample_size, test_type, nb_repeat)
        results["p_values_{}".format(sample_size)] = pvals
        
        # return the value of the statistic only for the chi2 test
        if test_type == "chi2":
            results["T_obs_{}".format(sample_size)] = T_obs    
    
    return results

In [ ]:
sizes = [10,30,50,100,200]
sample_size = 100
p1 = [0.4,0.6]
p2 = [0.1,0.9,0.5,0.5]
result_notindep_fisher = repeat_experience_several_sample_size(sizes, 
                                                        simu_sample_notindep_Y1_Y2, 
                                                        p1,
                                                        p2,
                                                        "fisher")

In [ ]:
result_notindep_pvals_fisher = pd.DataFrame(result_notindep_fisher).transpose()

In [ ]:
result_notindep_pvals_fisher

In [ ]:
plt.boxplot(result_notindep_pvals_fisher)
plt.xticks([1,2,3,4,5], sizes, fontsize = 15)
plt.yticks(fontsize = 15)
plt.ylabel("p-value", fontsize = 15)
plt.xlabel("Taille des échantillons", fontsize = 15)
plt.title("Répartition des p-value pour un test d'indépendance du $\chi2$ \n\
            pour 1000 échantillons\n\
            en fonction de la taille des échantillons", fontsize = 15)
plt.plot((0,7),(0.05,0.05))
plt.show()

# Get more on my github <img src="../images/github.png" width="100">
https://github.com/JJublanc/statistics_tools